In [4]:
# Import Libraries
import keras
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from IPython.display import FileLink, FileLinks
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import pylab
from scipy.stats.stats import pearsonr
%matplotlib inline

In [5]:
# Define to_image function
pixels = ["pixel_{0}".format(i) for i in range(20*20)]
def to_image(df):
    return  np.expand_dims(np.expand_dims(df[pixels], axis=-1).reshape(-1,20,20), axis=-1)

# List data files
datafilenames = ["SharedHits/NormalizedCharge/output_final_1.h5","SharedHits/AbsoluteCharge/output_final_1.h5",
                "Shared100NonShared/NormalizedCharge/output_final_1.h5","Shared100NonShared/AbsoluteCharge/output_final_1.h5",
                "AllHits/NormalizedCharge/output_final_1.h5","AllHits/AbsoluteCharge/output_final_1.h5"]
# Import data file
counter = 5
datafilename = datafilenames[counter]
df = pd.read_hdf("/uscms_data/d3/bbonham/TrackingDstar/LambaAnalyzer/output_of_postprocess/"+datafilename, 
                 key="df", mode='r', start=0, stop=10**5)

# Setup different cuts for different files
if counter == 0 or counter == 1: # Shared Hits
    cuttitle = "nUniqueSimTracksInSharedHit >= 0"
    stringforcut = "(df['nUniqueSimTracksInSharedHit']>=0)"
if counter == 2 or counter == 3: # Shared100NonShared Hits
    cuttitle = "nUniqueSimTracksInSharedHit != (0|1)"
    stringforcut = '(df["nUniqueSimTracksInSharedHit"]!=0) & (df["nUniqueSimTracksInSharedHit"]!=1)'
if counter == 4 or counter == 5: # All Hits
    cuttitle = "nUniqueSimTracksInSharedHit != (0|1)"
    stringforcut = '(df["nUniqueSimTracksInSharedHit"]!=0) & (df["nUniqueSimTracksInSharedHit"]!=1)'

# Copy of data set before making final cuts
df_old = df
df_old_train=df.sample(frac=0.5)
df_old_test=df.drop(df_old_train.index)
images_old_train = to_image(df_old_train)
images_old_test = to_image(df_old_test)

# Compute the pixel_number variable
pixelColumns = ["pixel_%i" % x for x in range(400)]
pixels_df = df[pixelColumns].values
pixel_number = pixels_df.astype(bool).sum(axis=1)
df.insert(0, "Pixel_number", pixel_number)

# Print info on dataframe
print '\n\033[1m' + datafilename[:-18] + '\033[0m' # file name printed in bold
print df.info()

# Make final cuts
df = df[(df["Pixel_number"]>1) & (df["GenDeltaR"]<0.1) & eval(stringforcut)]

# Print info on dataframe
print '\n\033[1m' + datafilename[:-18] + '\033[0m' # file name printed in bold
print df.info()
print ""


AllHits/AbsoluteCharge
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 408 entries, Pixel_number to pixel_148
dtypes: float32(1), float64(405), int32(1), int64(1)
memory usage: 313.8 MB
None

AllHits/AbsoluteCharge
<class 'pandas.core.frame.DataFrame'>
Int64Index: 22919 entries, 4 to 99988
Columns: 408 entries, Pixel_number to pixel_148
dtypes: float32(1), float64(405), int32(1), int64(1)
memory usage: 71.3 MB
None



In [6]:
df[df["totalCharge"]>0]

,Pixel_number,pixel_78,pixel_79,pixel_76,pixel_77,pixel_74,pixel_75,pixel_72,pixel_73,pixel_70,...,pixel_141,pixel_140,pixel_143,pixel_142,pixel_145,pixel_144,pixel_147,pixel_146,pixel_149,pixel_148
